

## Task A. MongoDB Data Model

### 1. Designing a suitable Data Model

To proceed with the model selection it is important to understand the two datasets Climate_historic.csv and hotspot_historic.csv.

Keen Observation determines the fundamental relationship between Climate  and Hotspot data along with the cardinality between the two. <br>

Relationship is as follows:<br>

<u>Climate Data</u><br>
Primary key:
- Date

<u>Hotspot Data</u><br>
Primary key:
- Latitude<br>
- Longitude<br>
- Datetime<br>

Foreign key:
- Date

The Caridnality between the two datasets :<br>
'Date' in Hotspot data (many) is a foreign key to 'Date' in  Climate data (one).<br>

There are two models mainly which can be used in case of one to many relationship, the relational or the reference model and the embedded model. Relational model or reference model where we create two collections one for climate data and the other for fire data. The fire id which will be added as primary key in fire data, acts as foreign key in the climate data.<br>

The model which we is using for building our model is embedded model where the climate record and the fire record will be embedded into a single document. For the quicker access of both climate and fire data, the embedded model gives flexibility during querying.


Sample Embedded Database would like as below:<br>

<u>'Embedded Climate-Fires'</u>

{ '_id': ObjectId('5b0111b8b82a9a1b30c8df32')
  'Air_Temperature_Celsius': 20,<br>
 'Date': '2017-03-06',<br>
 'Max_Wind_Speed': 21.0,<br>
 'Precipitation': ' 0.00I',<br>
 'Relative_Humidity': 52.9,<br>
 'Station': 948701,<br>
 'WindSpeed_knots': 17.0,<br>
 'Fire_record': {   'Confidence': 85,<br>
                     'Date': '2017-03-06',<br>
                     'Datetime': '2017-03-06T05:06:20',<br>
                     'Latitude': -34.3735,<br>
                     'Longitude': 141.6604,<br>
                     'Surface_Temperature_Celsius': 59,<br> }
}
  


The most optimal model to explain the one to many relationships is embedded schema with the hotspot data embedded with the climate data.<br>

We have chosen this approach of modelling for few specific reasons<br>
1. The Structure of the data is not complex.
2. The main aim of the project is identify the fire locations and datetime of occurances and hence we may need query fire records for a particular date frequently and hence by this approach it reduces the effort in querying. For instance we can just project all fire records without using a join. 
    

### Task B- Querying MongoDB using PyMongo

In [1]:
# Importing Required Libraries
import pandas as pd

In [2]:
# Loading historic data
climate = pd.read_csv("climate_historic.csv")
hotspot = pd.read_csv("hotspot_historic.csv")

In [3]:
climate.head()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation
0,948700,31/12/2016,19,56.8,7.9,11.1,0.00I
1,948700,2/01/2017,15,50.7,9.2,13.0,0.02G
2,948700,3/01/2017,16,53.6,8.1,15.0,0.00G
3,948700,4/01/2017,24,61.6,7.7,14.0,0.00I
4,948700,5/01/2017,24,62.3,7.0,13.0,0.00I


In [4]:
climate.shape

(366, 7)

Climate.csv has got 366 rows and 7 columns  

In [5]:
#Checking for the data types
climate.dtypes

station                      int64
date                        object
air_temperature_celcius      int64
relative_humidity          float64
windspeed_knots            float64
max_wind_speed             float64
precipitation               object
dtype: object

In [6]:
len(climate['date'].unique())

366

This shows that  dates are unique in climate_historic dataset and hence it acts as primary key for this table.

Lets look at the hotspot dataset as well.

In [7]:
hotspot.head()

,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
0,-37.966,145.051,2017-12-27T04:16:51,78,27/12/2017,68
1,-35.541,143.311,2017-12-27T00:02:15,82,27/12/2017,63
2,-35.554,143.307,2017-12-27T00:02:15,67,27/12/2017,53
3,-35.543,143.316,2017-12-27T00:02:14,86,27/12/2017,67
4,-37.708,145.100,2017-12-25T04:29:08,80,25/12/2017,54


In [8]:
#Cheking for Datatypes
hotspot.dtypes

latitude                       float64
longitude                      float64
datetime                        object
confidence                       int64
date                            object
surface_temperature_celcius      int64
dtype: object

In [9]:
# Splitting the date time component into two parts
hotspot[['date1','time']] = hotspot['datetime'].str.split('T', expand=True, n=1)
hotspot=hotspot.drop(['date1'], axis=1)

In [10]:
#Converting to correspoding datatyes
climate['date'] = pd.to_datetime(climate['date'])
hotspot['datetime'] = pd.to_datetime(hotspot['datetime'])
hotspot['date'] = pd.to_datetime(hotspot['date'])

In [11]:
# Sorting the dataframe by date for merging the fire and climate record
climate_sorted = climate.sort_values("date",ascending=False)
climate_sorted = climate_sorted.reset_index(drop=True)
hotspot_sorted = hotspot.sort_values("date",ascending=False)
hotspot_sorted = hotspot_sorted.reset_index(drop=True)

Merging of the two dataframes and converting the records into documents which consists of climate and fire records.
The k-way merge kind of algorithm is used to acheive the task.

In [12]:
j = 0
i = 0
final = {}
kinal = {}
climate_dict = {}
fire_data = {}
fire_list = []
total_list = []
while i < len(climate_sorted):
    if(j < len(hotspot_sorted)):
        if(climate_sorted['date'][i] == hotspot_sorted['date'][j]):
            final = {}
            final['latitude'] = hotspot_sorted['latitude'][j]
            final['longitude'] = hotspot_sorted['longitude'][j]
            final['datetime'] = str(hotspot_sorted['datetime'][j])
            final['confidence'] = int(hotspot_sorted['confidence'][j])
            final['surface_temperature_celcius'] = int(hotspot_sorted['surface_temperature_celcius'][j])
            final['time'] = hotspot_sorted['time'][j]
            
            fire_list.append(final)
            j = j + 1
        else:
            if(len(fire_list) != 0):
                climate_dict['fire_record']=fire_list
                fire_list = []
                climate_dict['station'] = int(climate_sorted['station'][i])
                climate_dict['air_temperature_celcius'] = int(climate_sorted['air_temperature_celcius'][i])
                climate_dict['relative_humidity'] = climate_sorted['relative_humidity'][i]
                climate_dict['windspeed_knots'] = climate_sorted['windspeed_knots'][i]
                climate_dict['max_wind_speed'] = climate_sorted['max_wind_speed'][i]
                climate_dict['precipitation'] = climate_sorted['precipitation '][i]
                climate_dict['date']= str(climate_sorted['date'][i].date())
                total_list.append(climate_dict)
            else:
                climate_dict['station'] = int(climate_sorted['station'][i])
                climate_dict['air_temperature_celcius'] = int(climate_sorted['air_temperature_celcius'][i])
                climate_dict['relative_humidity'] = climate_sorted['relative_humidity'][i]
                climate_dict['windspeed_knots'] = climate_sorted['windspeed_knots'][i]
                climate_dict['max_wind_speed'] = climate_sorted['max_wind_speed'][i]
                climate_dict['precipitation'] = climate_sorted['precipitation '][i]
                climate_dict['date']= str(climate_sorted['date'][i].date())
                total_list.append(climate_dict)
            climate_dict = {}
            final = {}
            kinal = {}
            fire_list = []
            i = i + 1
    else:
        climate_dict['station'] = int(climate_sorted['station'][i])
        climate_dict['air_temperature_celcius'] = int(climate_sorted['air_temperature_celcius'][i])
        climate_dict['relative_humidity'] = climate_sorted['relative_humidity'][i]
        climate_dict['windspeed_knots'] = climate_sorted['windspeed_knots'][i]
        climate_dict['max_wind_speed'] = climate_sorted['max_wind_speed'][i]
        climate_dict['precipitation'] = climate_sorted['precipitation '][i]
        climate_dict['date']= str(climate_sorted['date'][i].date())
        kinal = {}
        final = {}
        total_list.append(climate_dict)
        climate_dict = {}
        i = i +1

In [13]:
#Connecting to Mongo Db to insert the list od dictionaries to Database
import pymongo
from pymongo import MongoClient
client = MongoClient () # intialising the client

In [14]:
db = client.fit5148_db # Creating new Database
#Dropping the collections if already exists
if(db.climate.count() > 0):
    db.climate.drop()
test = db.climate # Creating new collection 
test.insert_many(total_list) # inserting records

### Part B2.A   

#### Find climate data on 10th December 2017

In [15]:
results = db.climate.aggregate([{"$match":{"date":"2017-12-10"}}])

In [16]:
for each in results:
    print(each)

{'max_wind_speed': 14.0, '_id': ObjectId('5ce79bcb9343692a45ae5422'), 'air_temperature_celcius': 15, 'windspeed_knots': 10.2, 'station': 948702, 'date': '2017-12-10', 'relative_humidity': 44.5, 'precipitation': ' 0.00G'}


### Part B2.B  

#### Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C

In [17]:
results = db.climate.aggregate([{"$unwind":"$fire_record"},
                            {"$match":{"fire_record.surface_temperature_celcius":{"$lt":100,"$gt":65}}},{
                                "$project":{"_id":0,"confidence":"$fire_record.confidence","latitude":"$fire_record.latitude","longitude":"$fire_record.longitude","surface_temperature_celcius":"$fire_record.surface_temperature_celcius"}
                            }])

In [18]:
for each in results:
    print(each)

{'latitude': -37.966, 'confidence': 78, 'longitude': 145.05100000000002, 'surface_temperature_celcius': 68}
{'latitude': -35.543, 'confidence': 86, 'longitude': 143.316, 'surface_temperature_celcius': 67}
{'latitude': -37.624, 'confidence': 90, 'longitude': 149.314, 'surface_temperature_celcius': 66}
{'latitude': -38.056999999999995, 'confidence': 93, 'longitude': 144.211, 'surface_temperature_celcius': 73}
{'latitude': -37.613, 'confidence': 95, 'longitude': 149.305, 'surface_temperature_celcius': 75}
{'latitude': -37.875, 'confidence': 93, 'longitude': 142.51, 'surface_temperature_celcius': 73}
{'latitude': -37.95, 'confidence': 92, 'longitude': 142.366, 'surface_temperature_celcius': 70}
{'latitude': -36.94, 'confidence': 89, 'longitude': 143.281, 'surface_temperature_celcius': 69}
{'latitude': -37.4671, 'confidence': 93, 'longitude': 145.9962, 'surface_temperature_celcius': 73}
{'latitude': -36.4489, 'confidence': 93, 'longitude': 144.7685, 'surface_temperature_celcius': 73}
{'lati

{'latitude': -36.1122, 'confidence': 90, 'longitude': 142.1491, 'surface_temperature_celcius': 76}
{'latitude': -37.5012, 'confidence': 100, 'longitude': 142.5853, 'surface_temperature_celcius': 90}
{'latitude': -34.351, 'confidence': 99, 'longitude': 141.6567, 'surface_temperature_celcius': 86}
{'latitude': -37.9247, 'confidence': 97, 'longitude': 142.8999, 'surface_temperature_celcius': 81}
{'latitude': -35.0805, 'confidence': 97, 'longitude': 142.4642, 'surface_temperature_celcius': 81}
{'latitude': -37.9452, 'confidence': 97, 'longitude': 143.5834, 'surface_temperature_celcius': 80}
{'latitude': -36.4557, 'confidence': 92, 'longitude': 142.2434, 'surface_temperature_celcius': 71}
{'latitude': -36.8009, 'confidence': 95, 'longitude': 142.7259, 'surface_temperature_celcius': 76}
{'latitude': -36.8099, 'confidence': 93, 'longitude': 142.72799999999998, 'surface_temperature_celcius': 73}
{'latitude': -36.7806, 'confidence': 94, 'longitude': 141.5597, 'surface_temperature_celcius': 74}


### Part B2.C

#### Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2017.


In [19]:
results = db.climate.aggregate([{"$unwind":"$fire_record"},
                            {"$match":{"date":{"$in":["2017-12-16","2017-12-15"]}}},{
                                "$project":{"_id":0,"date":1,"air_temperature_celcius":1,"relative_humidity":1,"max_wind_speed":1,"surface_temperature_celcius":"$fire_record.surface_temperature_celcius",}
                            }])

In [20]:
for each in results:
    print(each)

{'max_wind_speed': 13.0, 'relative_humidity': 53.7, 'date': '2017-12-16', 'air_temperature_celcius': 18, 'surface_temperature_celcius': 66}
{'max_wind_speed': 13.0, 'relative_humidity': 53.7, 'date': '2017-12-16', 'air_temperature_celcius': 18, 'surface_temperature_celcius': 57}
{'max_wind_speed': 13.0, 'relative_humidity': 53.7, 'date': '2017-12-16', 'air_temperature_celcius': 18, 'surface_temperature_celcius': 55}
{'max_wind_speed': 13.0, 'relative_humidity': 53.7, 'date': '2017-12-16', 'air_temperature_celcius': 18, 'surface_temperature_celcius': 48}
{'max_wind_speed': 13.0, 'relative_humidity': 53.7, 'date': '2017-12-16', 'air_temperature_celcius': 18, 'surface_temperature_celcius': 73}
{'max_wind_speed': 13.0, 'relative_humidity': 53.7, 'date': '2017-12-16', 'air_temperature_celcius': 18, 'surface_temperature_celcius': 60}
{'max_wind_speed': 13.0, 'relative_humidity': 53.7, 'date': '2017-12-16', 'air_temperature_celcius': 18, 'surface_temperature_celcius': 56}
{'max_wind_speed': 1

### Part B2.D

#### Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.


In [21]:
results = db.climate.aggregate([{"$unwind":"$fire_record"},
                            {"$match":{"fire_record.confidence":{"$gt":80,'$lt':100}}},{
                                "$project":{"_id":0,"datetime":"$fire_record.datetime","air_temperature_celcius":1,"surface_temperature_celcius":"$fire_record.surface_temperature_celcius","confidence":"$fire_record.confidence"}
                            }])

In [22]:
for each in results:
    print(each)

{'datetime': '2017-12-27 00:02:14', 'confidence': 86, 'air_temperature_celcius': 28, 'surface_temperature_celcius': 67}
{'datetime': '2017-12-27 00:02:15', 'confidence': 82, 'air_temperature_celcius': 28, 'surface_temperature_celcius': 63}
{'datetime': '2017-12-16 04:34:57', 'confidence': 90, 'air_temperature_celcius': 18, 'surface_temperature_celcius': 66}
{'datetime': '2017-12-16 00:21:11', 'confidence': 82, 'air_temperature_celcius': 18, 'surface_temperature_celcius': 55}
{'datetime': '2017-12-16 00:21:12', 'confidence': 93, 'air_temperature_celcius': 18, 'surface_temperature_celcius': 73}
{'datetime': '2017-12-16 00:20:53', 'confidence': 82, 'air_temperature_celcius': 18, 'surface_temperature_celcius': 64}
{'datetime': '2017-12-16 04:34:58', 'confidence': 95, 'air_temperature_celcius': 18, 'surface_temperature_celcius': 75}
{'datetime': '2017-12-16 04:34:58', 'confidence': 84, 'air_temperature_celcius': 18, 'surface_temperature_celcius': 55}
{'datetime': '2017-12-16 04:35:13', 'con

{'datetime': '2017-05-10 04:08:10', 'confidence': 81, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 54}
{'datetime': '2017-05-10 04:08:50', 'confidence': 82, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 55}
{'datetime': '2017-05-10 04:08:10', 'confidence': 96, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 78}
{'datetime': '2017-05-10 04:08:10', 'confidence': 99, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 84}
{'datetime': '2017-05-10 04:11:30', 'confidence': 83, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 57}
{'datetime': '2017-05-10 04:12:50', 'confidence': 86, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 60}
{'datetime': '2017-05-10 04:12:50', 'confidence': 87, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 63}
{'datetime': '2017-05-10 04:14:20', 'confidence': 88, 'air_temperature_celcius': 10, 'surface_temperature_celcius': 64}
{'datetime': '2017-05-10 04:16:10', 'con

{'datetime': '2017-04-13 04:26:30', 'confidence': 97, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 80}
{'datetime': '2017-04-13 04:26:30', 'confidence': 86, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 60}
{'datetime': '2017-04-13 04:26:30', 'confidence': 87, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 61}
{'datetime': '2017-04-13 04:26:30', 'confidence': 81, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 55}
{'datetime': '2017-04-13 04:26:30', 'confidence': 88, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 63}
{'datetime': '2017-04-13 04:26:30', 'confidence': 81, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 54}
{'datetime': '2017-04-13 04:26:30', 'confidence': 92, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 70}
{'datetime': '2017-04-13 04:26:30', 'confidence': 87, 'air_temperature_celcius': 16, 'surface_temperature_celcius': 63}
{'datetime': '2017-04-13 04:26:30', 'con

### Part B2.E

#### Find the top 10 records with the highest surface temperature (°C)

In [23]:
results = db.climate.aggregate([{"$unwind":"$fire_record"},{"$sort":{"fire_record.surface_temperature_celcius":-1}},{"$limit":10},{"$project":{"_id":0}}])

In [24]:
for each in results:
    print(each)

{'max_wind_speed': 9.9, 'air_temperature_celcius': 15, 'windspeed_knots': 5.1, 'station': 948701, 'date': '2017-04-18', 'relative_humidity': 56.1, 'fire_record': {'datetime': '2017-04-18 04:52:00', 'surface_temperature_celcius': 124, 'latitude': -38.1665, 'longitude': 143.062, 'confidence': 100, 'time': '04:52:00'}, 'precipitation': ' 0.00I'}
{'max_wind_speed': 12.0, 'air_temperature_celcius': 16, 'windspeed_knots': 5.4, 'station': 948701, 'date': '2017-04-04', 'relative_humidity': 47.5, 'fire_record': {'datetime': '2017-04-04 04:32:50', 'surface_temperature_celcius': 123, 'latitude': -36.343, 'longitude': 142.1986, 'confidence': 100, 'time': '04:32:50'}, 'precipitation': ' 0.00I'}
{'max_wind_speed': 20.0, 'air_temperature_celcius': 14, 'windspeed_knots': 10.5, 'station': 948701, 'date': '2017-01-05', 'relative_humidity': 49.2, 'fire_record': {'datetime': '2017-05-01 04:14:20', 'surface_temperature_celcius': 122, 'latitude': -36.9318, 'longitude': 143.0907, 'confidence': 100, 'time': '

### Part B2.F

#### Find the number of fire in each day. You are required to only display the total number of fire and the date in the output

In [25]:
results=db.climate.aggregate([{"$unwind":"$fire_record"},{
    '$group':{
        '_id':'$date',
        'date':{"$first":"$date"},
        'Number of fires':{'$sum':1}
        
    }},
    {
    '$project':{'_id':0}}
])

In [26]:
for each in results:
    print(each)

{'date': '2017-01-06', 'Number of fires': 2}
{'date': '2017-01-07', 'Number of fires': 4}
{'date': '2017-02-04', 'Number of fires': 5}
{'date': '2017-02-08', 'Number of fires': 2}
{'date': '2017-03-04', 'Number of fires': 72}
{'date': '2017-03-05', 'Number of fires': 64}
{'date': '2017-03-13', 'Number of fires': 2}
{'date': '2017-03-15', 'Number of fires': 7}
{'date': '2017-03-18', 'Number of fires': 3}
{'date': '2017-03-19', 'Number of fires': 21}
{'date': '2017-03-24', 'Number of fires': 2}
{'date': '2017-03-28', 'Number of fires': 54}
{'date': '2017-03-29', 'Number of fires': 1}
{'date': '2017-04-04', 'Number of fires': 89}
{'date': '2017-04-05', 'Number of fires': 135}
{'date': '2017-04-07', 'Number of fires': 1}
{'date': '2017-04-13', 'Number of fires': 357}
{'date': '2017-01-08', 'Number of fires': 2}
{'date': '2017-04-14', 'Number of fires': 18}
{'date': '2017-04-15', 'Number of fires': 69}
{'date': '2017-04-17', 'Number of fires': 38}
{'date': '2017-04-18', 'Number of fires': 3

### Part B2.G

#### Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.

In [27]:
results=db.climate.aggregate([{"$unwind":"$fire_record"},{
    '$group':{
        '_id':'$date',
        'date':{"$first":"$date"},
        'Average surface temperature':{'$avg':"$fire_record.surface_temperature_celcius"}
        
    }},
    {
    '$project':{'_id':0}}
])

In [28]:
for each in results:
    print(each)

{'date': '2017-01-06', 'Average surface temperature': 54.0}
{'date': '2017-01-07', 'Average surface temperature': 30.5}
{'date': '2017-02-04', 'Average surface temperature': 45.2}
{'date': '2017-02-08', 'Average surface temperature': 63.5}
{'date': '2017-03-04', 'Average surface temperature': 58.44444444444444}
{'date': '2017-03-05', 'Average surface temperature': 56.796875}
{'date': '2017-03-13', 'Average surface temperature': 38.5}
{'date': '2017-03-15', 'Average surface temperature': 46.0}
{'date': '2017-03-18', 'Average surface temperature': 79.33333333333333}
{'date': '2017-03-19', 'Average surface temperature': 65.57142857142857}
{'date': '2017-03-24', 'Average surface temperature': 49.0}
{'date': '2017-03-28', 'Average surface temperature': 60.925925925925924}
{'date': '2017-03-29', 'Average surface temperature': 51.0}
{'date': '2017-04-04', 'Average surface temperature': 62.57303370786517}
{'date': '2017-04-05', 'Average surface temperature': 56.80740740740741}
{'date': '2017-0